# Sources in Roskilde Fjord

1. Extract coordinates of all sources from pfs file
2. Convert from local projected coordinate system to geographical coordinates
3. Plot the location of each source on a map

## Read the PFS file

In [1]:
from pfs2yaml import pfs2dict, pfs2yaml

pfsfilename = "RoskildeFjord_c9_2011_coarse.m3fm"

with open(pfsfilename) as f:
        pfs = f.read()

y = pfs2yaml(pfs)
d = pfs2dict(pfs)

## Get projection

In [18]:
wkt = d["FemEngineHD"]["DOMAIN"]['coordinate_type']
wkt

'PROJCS["ETRS_1989_Kp2000_Zealand",GEOGCS["GCS_ETRS_1989",DATUM["D_ETRS_1989",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",12.0],PARAMETER["Scale_Factor",0.99995],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]'

In [21]:
import pyproj

inProj = pyproj.CRS.from_string(wkt)
inProj

<CRS: PROJCS["ETRS_1989_Kp2000_Zealand",GEOGCS["GCS_ETRS ...>
Name: ETRS89 / Kp2000 Zealand
Ellipsoid:
- semi_major_metre: 6378137.00
- semi_minor_metre: 6356752.31
- inverse_flattening: 298.26
Area of Use:
- UNDEFINED
Prime Meridian:
- longitude: 0.0000
- unit_name: Degree
- unit_conversion_factor: 0.01745329
Axis Info:
- Easting[E] (east) EPSG:9001 (metre)
- Northing[N] (north) EPSG:9001 (metre)

## Read source data

In [44]:
from collections import namedtuple

Source = namedtuple('Source',['name','lon','lat'])

outProj = pyproj.Proj(init='epsg:4326')
sources = d["FemEngineHD"]["HYDRODYNAMIC_MODULE"]["SOURCES"]

n_sources = sources['number_of_sources']

out = []

for i in range(n_sources):

    key = f"SOURCE_{i+1}"
    
    source = sources[key]
    name = source['name']
    x,y,z = source['coordinates']
    
    lon,lat = pyproj.transform(inProj,outProj,x,y)
    s = Source(name=name,lon=lon,lat=lat)
    out.append(s)
    


In [45]:
out[0:2]

[Source(name='Arresø Kanal', lon=12.00640530070797, lat=55.96360022598667),
 Source(name='Havelse Å', lon=12.057519866061158, lat=55.877360558793754)]

## Create an interactive map

In [47]:
import folium
m = folium.Map(location=[55.8, 12])

for s in out:
    folium.Marker([s.lat, s.lon], tooltip=s.name).add_to(m)

m